# Weekly Counts of Deaths by Jurisdiction and Age

* Download the CDC data from https://data.cdc.gov/NCHS/Weekly-Counts-of-Deaths-by-Jurisdiction-and-Age/y5bj-9g5w
* Download the NYT COVID-19 Deaths from https://github.com/nytimes/covid-19-data
* Download population data from https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/state/asrh/
* Merge all into `data/us_excess_deaths.rds`

In [1]:
library(tidyverse)

options(readr.show_col_types = F)

download.file("https://data.cdc.gov/api/views/y5bj-9g5w/rows.csv?accessType=DOWNLOAD",
             destfile = "data/Weekly_Counts_of_Deaths_by_Jurisdiction_and_Age.csv")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Make one age bin of all age groups <65

In [2]:
df <- read_csv('data/Weekly_Counts_of_Deaths_by_Jurisdiction_and_Age.csv',
              col_types = "ccciicicccc")%>%
  filter(Type != 'Predicted (weighted)')%>%
  select(Jurisdiction, Year, Week, `Number of Deaths`, `Age Group`)%>%
  mutate (age_lower = parse_number(gsub("Under 25 years", "0", `Age Group`)))%>%
  mutate(`Age Group` = case_when(age_lower<65 ~ "<65",
                                 age_lower<75 ~ "65-74",
                                 age_lower<85 ~ "75-84",
                                            T ~ "85+"))%>%
  group_by(Jurisdiction, Year, Week, `Age Group`)%>%
  summarise(`Number of Deaths` = sum(ifelse(is.na(`Number of Deaths`), 0, `Number of Deaths`)))

head(df)

`summarise()` has grouped output by 'Jurisdiction', 'Year', 'Week'. You can
override using the `.groups` argument.


Jurisdiction,Year,Week,Age Group,Number of Deaths
<chr>,<int>,<int>,<chr>,<dbl>
Alabama,2015,1,<65,345
Alabama,2015,1,65-74,202
Alabama,2015,1,75-84,272
Alabama,2015,1,85+,320
Alabama,2015,2,<65,333
Alabama,2015,2,65-74,222


# Merge 'New York' and 'New York City'
In the CDC data 'New York' means 'New York *excluding* New York City'

In [3]:
df2 <-
  df%>%
  mutate(Jurisdiction = gsub("New York City", "New York", Jurisdiction))%>%
  group_by(Jurisdiction, Year, `Age Group`, Week)%>%
  summarise(`Number of Deaths` = sum(`Number of Deaths`))

write_csv(df2, "data/tidy_deaths_age_state_CDC.csv")

head(df2)

`summarise()` has grouped output by 'Jurisdiction', 'Year', 'Age Group'. You
can override using the `.groups` argument.


Jurisdiction,Year,Age Group,Week,Number of Deaths
<chr>,<int>,<chr>,<int>,<dbl>
Alabama,2015,<65,1,345
Alabama,2015,<65,2,333
Alabama,2015,<65,3,303
Alabama,2015,<65,4,294
Alabama,2015,<65,5,316
Alabama,2015,<65,6,291


# Download population data
See https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2020/sc-est2020-agesex-civ.pdf
for a description

In [4]:
download.file("https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/state/asrh/SC-EST2020-AGESEX-CIV.csv",
             destfile = "data/Population.csv")

In [5]:
df_pop <- read_csv('data/Population.csv')
head(df_pop)

SUMLEV,REGION,DIVISION,STATE,NAME,SEX,AGE,ESTBASE2010_CIV,POPEST2010_CIV,POPEST2011_CIV,POPEST2012_CIV,POPEST2013_CIV,POPEST2014_CIV,POPEST2015_CIV,POPEST2016_CIV,POPEST2017_CIV,POPEST2018_CIV,POPEST2019_CIV,POPEST2020_CIV
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
010,0,0,0,United States,0,0,3944160,3951495,3963264,3926731,3931411,3954973,3984144,3963268,3882437,3826908,3762227,3735010
010,0,0,0,United States,0,1,3978090,3957904,3966768,3978210,3943348,3949559,3973828,4003586,3981864,3897917,3842257,3773884
010,0,0,0,United States,0,2,4096939,4090799,3971498,3980139,3993047,3960015,3967672,3992657,4021261,3996742,3911822,3853025
010,0,0,0,United States,0,3,4119051,4111869,4102429,3983007,3992839,4007852,3976277,3984985,4009060,4035053,4009037,3921526
010,0,0,0,United States,0,4,4063186,4077511,4122252,4112849,3994539,4006407,4022785,3992241,4000394,4021907,4045996,4017847
010,0,0,0,United States,0,5,4056872,4064653,4087770,4132349,4123745,4007123,4020489,4038022,4007233,4012789,4032231,4054336


In [23]:
df_pop2 <-  
  df_pop%>%
  filter(SEX==0 & AGE != 999)%>%
  mutate(`Age Group` = case_when(AGE<65 ~ "<65",
                                 AGE<75 ~ "65-74",
                                 AGE<85 ~ "75-84",
                                      T ~ "85+"))%>%
  group_by(NAME, `Age Group`)%>%
  summarise(Population = sum(POPEST2020_CIV))%>%
  rename(Jurisdiction = NAME)

write_csv(df_pop2, "data/tidy_pop_age_state.csv")

head(df_pop2)

`summarise()` has grouped output by 'NAME'. You can override using the
`.groups` argument.


Jurisdiction,Age Group,Population
<chr>,<chr>,<dbl>
Alabama,<65,4033195
Alabama,65-74,517717
Alabama,75-84,263668
Alabama,85+,92859
Alaska,<65,615848
Alaska,65-74,64158


# Excess Deaths Associated with COVID-19
Download the CDC data from https://www.cdc.gov/nchs/nvss/vsrr/covid19/excess_deaths.htm

In [7]:
download.file("https://data.cdc.gov/api/views/xkkf-xrst/rows.csv?accessType=DOWNLOAD",
             destfile = "data/Excess_Deaths_Associated_with_COVID-19.csv")
head(df)

Jurisdiction,Year,Week,Age Group,Number of Deaths
<chr>,<int>,<int>,<chr>,<dbl>
Alabama,2015,1,<65,345
Alabama,2015,1,65-74,202
Alabama,2015,1,75-84,272
Alabama,2015,1,85+,320
Alabama,2015,2,<65,333
Alabama,2015,2,65-74,222


In [25]:
df <- 
  read_csv("data/Excess_Deaths_Associated_with_COVID-19.csv")%>%
  filter(Type=="Unweighted")%>%
  select(State, `Week Ending Date`, `Average Expected Count`, `Observed Number`)%>%
  mutate(State = gsub("New York City", "New York", State))%>%
  group_by(`Week Ending Date`, State)%>%
  summarise(`Average Expected Count` = sum(`Average Expected Count`),
           `Observed Number` = sum(`Observed Number`))

write_csv(df, "data/tidy_deaths_state_CDC.csv")

head(df)

`summarise()` has grouped output by 'Week Ending Date'. You can override using
the `.groups` argument.


Week Ending Date,State,Average Expected Count,Observed Number
<date>,<chr>,<dbl>,<dbl>
2017-01-07,Alabama,1034,1121
2017-01-07,Alaska,79,104
2017-01-07,Arizona,1118,1170
2017-01-07,Arkansas,635,651
2017-01-07,California,5197,6061
2017-01-07,Colorado,756,793


# Download COVID-19 deaths from the New York Times

In [26]:
df_covid <- 
  read_csv("https://github.com/nytimes/covid-19-data/raw/master/us-states.csv")%>%
  group_by(state)%>%
  arrange(date)%>%
  mutate(deaths = deaths-lag(deaths, 1, default = 0))%>%
  mutate(`Covid Deaths` = zoo::rollmean(deaths, k=21, na.pad = T))%>%
  rename(`Week Ending Date` = date,
         State = state)%>%
  select(`Week Ending Date`, State, `Covid Deaths`)%>%
  na.omit()%>%
  ungroup()

# Sum up all states for the 'United States' estimate
df_covid <- 
rbind.data.frame(df_covid,
  df_covid%>%
    group_by(`Week Ending Date`)%>%
    summarise(`Covid Deaths` = sum(`Covid Deaths`))%>%
    ungroup()%>%
    mutate(State = "United States")%>%
    select(`Week Ending Date`, `State`, `Covid Deaths`)
  )
tail(df_covid)

Week Ending Date,State,Covid Deaths
<date>,<chr>,<dbl>
2022-04-05,United States,566.5714
2022-04-06,United States,560.7143
2022-04-07,United States,559.1429
2022-04-08,United States,530.9048
2022-04-09,United States,516.4762
2022-04-10,United States,488.0476


In [27]:
df2 <-
  df%>%
  select(State, `Week Ending Date`, `Average Expected Count`, `Observed Number`)%>%
  merge(df_covid, all = T)%>%
  drop_na(`Observed Number`)%>%
  merge(
    read_csv("data/tidy_pop_age_state.csv", show_col_types = FALSE)%>%
      group_by(Jurisdiction)%>%
      summarise(Population = sum(Population))%>%
      rename(State = Jurisdiction)
  )

write_rds(df2, "data/us_excess_deaths.rds")

head(df2)

,State,Week Ending Date,Average Expected Count,Observed Number,Covid Deaths,Population
,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>
1,Alabama,2019-06-01,999,1027,NA,4907439
2,Alabama,2019-06-08,993,1029,NA,4907439
3,Alabama,2019-06-29,982,940,NA,4907439
4,Alabama,2019-07-06,984,1028,NA,4907439
5,Alabama,2019-06-15,988,986,NA,4907439
6,Alabama,2019-06-22,982,990,NA,4907439
